# Les Optimisations de type réseau - Partie 2

# Introduction.
Les optimisations de type réseaux ont cela en commun qu'elles mettent en jeu 
des noeuds (nodes) et des arrêtes valuées (arcs).

Dans cette étude, je vais tenter, une fois les optimisations classiques abordées,
d'étudier la possibilité d'y ajouter des paramètres non linéaires dans un second temps.


Etude globale proposée par <b> Estelle Derrien - Github estellederrien </b>

!! Création en cours - sujet à de lourdes modifications !!

# Sommaire

  
   - 1. <b>Problème de Flots</b> (Flows models)
         - Flots de coût minimal.
            - Description
            - Modélisation mathématique
            - Notre exemple de base
            - Solution avec le solveur Pulp
            - Solution avec NetWorkX
         - Flot maximal.
            - Description
            - Modélisation mathématique
            - Notre exemple de base
            - SOlution avec Google OR Tools
            - Solution avec le solveur Pulp
            - Solution avec l'algo de Ford Fulkerson

   - 2. <b>Plus court chemin</b> (Shortest path problem)
         - Description
         - Modélisation mathématique
         - Exemples avec les solveurs  
   - 3. <b> Problème de selection de projets et selection.
         - Description et lien
         - Modélisation mathématique
         - Exemple avec le solveur 

   - 4. <b>Optimisation de production à l'aide d'un modèle de type réseau.</b>
         - Description
         - Modélisation mathématique
         - Exemples avec les solveurs
   - 5. <b>Optimisation financière à l'aide d'un modèle de type réseau. ( Financial optimizations using networks models)</b>
         - Description
         - Modélisation mathématique
         - Exemples avec les solveurs

   - 6. <b>Optimisation mathématique en réseaux informatique (Computer networks optimizations)</b>
         - Description
         - Modélisation mathématique
         - Exemples avec les solveurs
   - 7. <b> Le problème du voyageur de commerce ( Traveling salesman)</b>
         - Description
         - Modélisation mathématique
         - Exemples avec les solveurs
         - Le concorde tsp Solver : https://www.math.uwaterloo.ca/tsp/concorde/
         - Résolution Avec un algorithme génétique : https://nbviewer.org/github/rhiever/Data-Analysis-and-Machine-Learning-Projects/blob/master/optimal-road-trip/Computing%20the%20optimal%20road%20trip%20across%20the%20U.S..ipynb
   - 8. <b> Coloration de graphe. </b>
         - Description - https://github.com/tpawelski/graph-coloring-lp
         - Modélisation mathématique
         - Exemples avec les solveurs      
   - 9. <b> Problèmes de livraison par véhicule</b>(Véhicules routing problems)
         - Variantes
   - 10. <b> Problèmes de partitionnement.

   - <b> Liens </b>


# 1. Les problèmes de flots

## <b> A/ Le problème de flot de coût minimal</b>

## Description.

Le problème du flux à coût minimum consiste à trouver le moyen le moins cher possible d’envoyer une certaine quantité de flux à travers un réseau de flux. 
La résolution de ce problème est utile pour les situations réelles impliquant des réseaux avec des coûts associés (par exemple les réseaux de télécommunications), ainsi que dans d'autres situations où l'analogie n'est pas si évidente, comme par exemple où localiser l'entrepôt.

https://fr.wikipedia.org/wiki/Probl%C3%A8me_du_flot_de_co%C3%BBt_minimum



## Modélisation mathématique


Étant donné un réseau  G  constitué de  n  sommets et de  m  arêtes. Pour chaque arête (en général, arêtes orientées), la capacité (un nombre entier non négatif) et le coût par unité de flux le long de cette arête (un nombre entier) sont donnés. De plus, la source  s  et la sortie finale t (sink en Anglais) sont marqués.

Pour une valeur donnée  K , il faut trouver un flux de cette quantité, et parmi tous les flux de cette quantité il faut choisir le flux ayant le coût le plus bas. Cette tâche est appelée problème de flux à coût minimum.



## Notre problème de base

Source : https://lpsolve.sourceforge.net/5.5/DIMACS_mcf.htm


<div style="text-align:center">
<img src="img/mincostflow.png">
</div>

- Les noeuds ont un identifiant
- Les arrêtes ont une capacité minimale, une capacité max , et un coût en dollars

Dans le graphique, le noeud 1 fournit 4 , et le noeud 4 a une demande de 4, les noeuds 2 et 3 ont une demande de 0.

## Solution avec Python Pulp

In [23]:
'''
Minimum Cost Flow Problem Solver with PuLP
Adapted from:
https://code.google.com/p/pulp-or/wiki/ATransshipmentProblem
The American Steel Problem for the PuLP Modeller
Authors: Antony Phillips, Dr Stuart Mitchell   2007
 
'''
 
# Import PuLP modeller functions
from pulp import *
 
# list of nodes
nodes = [1,2,3,4]
 
# supply or demand of nodes
            #NodeID : [Supply,Demand]
nodeData = {1:[4,0],
            2:[0,0],
            3:[0,0],
            4:[0,4]}
 
# arcs list
arcs = [ (1,2),
         (1,3),
         (2,3),
         (2,4),
         (3,4)]
 
# arcs cost, lower bound and capacity
            #Arc : [Cost,MinFlow,MaxFlow]
arcData = { (1,2):[2,0,4],
            (1,3):[2,0,2],
            (2,3):[1,0,2],
            (2,4):[3,0,3],
            (3,4):[1,0,5] }
 
# Splits the dictionaries to be more understandable
(supply, demand) = splitDict(nodeData)
(costs, mins, maxs) = splitDict(arcData)
 
# Creates the boundless Variables as Integers
vars = LpVariable.dicts("Route",arcs,None,None,LpInteger)
 
# Creates the upper and lower bounds on the variables
for a in arcs:
    vars[a].bounds(mins[a], maxs[a])
 
# Creates the 'prob' variable to contain the problem data    
prob = LpProblem("Minimum_Cost_Flow_Problem_Sample",LpMinimize)
 
# Creates the objective function
prob += lpSum([vars[a]* costs[a] for a in arcs]), "Total Cost of Transport"
 
# Creates all problem constraints - this ensures the amount going into each node is 
# at least equal to the amount leaving
for n in nodes:
    prob += (supply[n]+ lpSum([vars[(i,j)] for (i,j) in arcs if j == n]) >=
             demand[n]+ lpSum([vars[(i,j)] for (i,j) in arcs if i == n])), \
            "Flow Conservation in Node %s"%n
 
# The problem data is written to an .lp file
prob.writeLP("simple_MCFP.lp")
 
# The problem is solved using PuLP's choice of Solver
prob.solve()


# On imprime le résultat
print('Statut:', LpStatus[prob.status])
print('Coût total de flot minimisé = ', value(prob.objective))
 
for i in prob.variables():
    if i.varValue > 0:
        print('Flot :',i.name, '=', i.varValue)

c:\Users\secretgirl\AppData\Local\Programs\Python\Python38\lib\site-packages\pulp\pulp.py:1195: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Statut: Optimal
Coût total de flot minimisé =  14.0
Flot : Route_(1,_2) = 2.0
Flot : Route_(1,_3) = 2.0
Flot : Route_(2,_3) = 2.0
Flot : Route_(3,_4) = 4.0



## <b> B/ Le problème de flot maximal</b>

## Description.

Dans la théorie de l'optimisation, les problèmes de débit maximum impliquent de trouver un débit réalisable à travers un réseau de flux qui obtient le débit maximum possible.
Les arrêtes(arcs) ont une capacité de flot maximum, mais plusieurs routes peuvent exister, plus ou moins avantageuses .
Le problème de flot maximum consiste à trouver, dans un réseau de flot, un flot réalisable depuis une source unique et vers un puits unique qui soit maximum1. 

https://fr.wikipedia.org/wiki/Probl%C3%A8me_de_flot_maximum
https://en.wikipedia.org/wiki/Maximum_flow_problem

https://www.excel-easy.com/examples/maximum-flow-problem.html

https://medium.com/@nikunjsaini37/linear-programming-approach-to-solve-the-maximum-flow-problem-407f29cbe407

## Modélisation mathématique

Étant donné un graphe orienté G = ( V , E ) G=(V,E), où chaque arc u , v u,v a une capacité c ( u , v ) c(u,v), on cherche un flot maximum f f depuis la source s s vers le puits t t, sous contrainte de capacité.

On maximise
\sum _{{v\in V}}f(s,v)

## Notre problème de base

Pour tester nos code python de résolution de problème de flots maximum, on va prendre le problème exposé par le site excel easy : 
https://www.excel-easy.com/examples/maximum-flow-problem.html

<div style="text-align:center">
<img src="img/maxflow.png">
</div>

Sauf qu'on ne va pas résoudre avec le solveur Excel, mais avec des solveurs Python et vérifier que le résultat est identique.



## Solution avec Google Or

On utilise ce code : 
https://developers.google.com/optimization/flow/maxflow?hl=fr

On voit que le résultat est le même que celui de EXCEL

In [24]:
"""From Taha 'Introduction to Operations Research', example 6.4-2."""
import numpy as np
from ortools.graph.python import max_flow


def main():
    """MaxFlow simple interface example."""
    # Instantiate a SimpleMaxFlow solver.
    smf = max_flow.SimpleMaxFlow()

    # Define three parallel arrays: start_nodes, end_nodes, and the capacities
    # between each pair. For instance, the arc from node 0 to node 1 has a
    # capacity of 20.
    # Note estelle : Ici on entre les neuds comme dans le pb EXCEL vu plus haut.

    # Table de conversion pour que cela soit comme Excel, l'algo n'accepte pas les strings:
    # Node S = 0
    # Node A = 1
    # Node B = 2
    # Node C = 3
    # Node D = 4
    # Node E = 5
    # Node T = 6

    # On crée les arcs et les capacités max
    start_nodes = np.array([0,0,0,1,1,2,2,3,3,3,4,5])
    end_nodes = np.array([  1,2,3,3,4,3,5,4,5,6,6,6])
    capacities = np.array([ 4,2,8,5,2,6,9,1,3,4,7,5])

    # Add arcs in bulk.
    #   note: we could have used add_arc_with_capacity(start, end, capacity)
    all_arcs = smf.add_arcs_with_capacity(start_nodes, end_nodes, capacities)

    # EXEMPLE INITIAL : Find the maximum flow between node 0 and node 4.
    # Trouver le fot maximum entre node  0 et 6
    status = smf.solve(0, 6)

    if status != smf.OPTIMAL:
        print("There was an issue with the max flow input.")
        print(f"Status: {status}")
        exit(1)
    print("Max flow:", smf.optimal_flow())
    print("")
    print(" Arc    Flow / Capacity")
    solution_flows = smf.flows(all_arcs)
    for arc, flow, capacity in zip(all_arcs, solution_flows, capacities):
        print(f"{smf.tail(arc)} / {smf.head(arc)}   {flow:3}  / {capacity:3}")
    print("Source side min-cut:", smf.get_source_side_min_cut())
    print("Sink side min-cut:", smf.get_sink_side_min_cut())


if __name__ == "__main__":
    main()

Max flow: 12

 Arc    Flow / Capacity
0 / 1     2  /   4
0 / 2     2  /   2
0 / 3     8  /   8
1 / 3     0  /   5
1 / 4     2  /   2
2 / 3     0  /   6
2 / 5     2  /   9
3 / 4     1  /   1
3 / 5     3  /   3
3 / 6     4  /   4
4 / 6     3  /   7
5 / 6     5  /   5
Source side min-cut: [0, 1, 3]
Sink side min-cut: [6, 4]


## Solution avec Python Pulp

Le Modèle ( En Anglais )


<div style="text-align:center">
<img src="img/maxflowmodel.jpg">
</div>


In [25]:
# Code à venir

## L'algorithme de Ford Fulkerson avec Python

Code à venir

# 3. Problème de selection de projets et sélection de machines.

## Description et lien

Dans le problème de sélection de projet, il y a n projets et m machines. Chaque projet pi génère un revenu r(pi) et chaque machine qj coûte c(qj) à l'achat. Chaque projet nécessite un certain nombre de machines et chaque machine peut être partagée par plusieurs projets. Le problème est de déterminer quels projets et quelles machines doivent être sélectionnés et achetés respectivement, afin que le profit soit maximisé.

You are given a set of N projects and M machines. The i-th machine costs qi. The i-th project yields pi revenue. Each project requires a set of machines. If multiple projects require the same machine, they can share the same one. Choose a set of machines to buy and projects to complete such that the sum of the revenues minus the sum of the costs is maximized."



Lien :
https://en.wikipedia.org/wiki/Max-flow_min-cut_theorem

https://codeforces.com/blog/entry/101354

https://ocw.tudelft.nl/wp-content/uploads/Algoritmiek_Project_Selection.pdf

Min cut :
https://www.scaler.com/topics/data-structures/maximum-flow-and-minimum-cut/


# 8. Problèmes de livraison par véhicules

## Variantes:

Il existe de nombreuses variantes du problème de tournées de véhicules. Voici quelques exemples populaires :

     CVRP (Capacitated Vehicle Routing Problem) : Les véhicules ont une capacité de charge limitée des marchandises qui doivent être livrées.

     VRPTW (Vehicle Routing Problem with Time Windows) : Les lieux de livraison ont des plages horaires dans lesquelles les livraisons (ou visites) doivent être effectuées.

     VRPPD (Vehicle Routing Problem with Pickup and Delivery) : Un certain nombre de marchandises doivent être déplacées de certains lieux de ramassage vers d'autres lieux de livraison. L'objectif est de trouver des itinéraires optimaux pour qu'une flotte de véhicules visite les lieux de prise en charge et de dépose.
     
     VRPP (Vehicle Routing Problem with Profits) : C'est un problème de maximisation où il n'est pas obligatoire pour les véhicules de visiter tous les nœuds. L'objectif est de visiter les nœuds pour maximiser la somme des bénéfices collectés dans des circonstances spécifiques tout en respectant une limite de temps de véhicule donnée. Par exemple, supposons qu'il existe deux ensembles de clients, les uns sont des clients fréquents qui sont obligatoires pour la livraison, et un autre ensemble est les clients potentiels non fréquents avec des bénéfices connus et estimés. Les deux ensembles ont des demandes et des exigences de service connues sur un horizon de planification de plusieurs jours. L’objectif est de déterminer les tournées des véhicules qui maximisent le profit net, tout en satisfaisant les contraintes de capacité des véhicules, de durée de tournée et de cohérence. Les véhicules doivent commencer et finir au même dépôt.

## Liens :

https://developers.google.com/optimization/routing/vrptw?hl=fr

https://github.com/google/or-tools/blob/stable/examples/python/prize_collecting_vrp_sat.py

https://or.stackexchange.com/questions/10711/team-orienteering-problem-with-time-windows-infeasible-using-cp-sat-solver-in-go

https://github.com/google/or-tools/blob/stable/examples/python/prize_collecting_tsp_sat.py

http://alvarestech.com/temp/vrptw/Vehicle%20Routing%20Problem%20with%20Time%20Windows.pdf

https://www.linkedin.com/pulse/vehicle-routing-problem-pulp-real-world-scenarios-dhawal-thakkar/

https://medium.com/jdsc-tech-blog/capacitated-vehicle-routing-problem-cvrp-with-python-pulp-and-google-maps-api-5a42dbb594c0

https://how-to.aimms.com/Articles/332/332-Miller-Tucker-Zemlin-formulation.html

